In [1]:
# !pip install orix
# Uncomment the above line when running the code file for the first time to ensure that your system has orix installed

In [1]:
import tkinter as tk
from PIL import ImageTk, Image

%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from orix import plot, sampling
from orix.crystal_map import Phase
from orix.quaternion import Orientation, symmetry
from orix.vector import Vector3d

# Ensure that you have the above libraries installed else it will throw an error
# General suggestion is to try using '!pip install keyword' where keyword needs to be replaced with library name
# keyword can be matplotlib, numpy etc

# Changing parameters to change plots characteristics
new_params = {
    "figure.facecolor": 'w',    # changes background color of image
    "figure.figsize": (5, 3),   # changes figure size of image
    "lines.markersize": 0,      # changes size of marker
    "font.size": 6,             # changes font size
    "axes.grid": False,         # controls the visibility of grid lines in a plot
}
plt.rcParams.update(new_params)

plot.IPFColorKeyTSL(symmetry.Oh).plot()    # draws IPF color key D4h Schoenflies and 4/mmm Laue class
# please refer to orix documentation for other symmetries
# paste the below link on your browser to jump to orix documentation
# https://orix.readthedocs.io/en/stable/tutorials/inverse_pole_figures.html

plt.savefig('IPF1.png')          # saves the IPF color key image for further use
# if you want to change image file name - change 'IPF.png' to 'XXXX.png' above where XXXX is the image file name
# if you want to save the image file to a different location then replace 'IPF.png' with the path above 

In [ ]:
import tkinter as tk
from PIL import Image, ImageTk, ImageDraw
import math

# Function to compare RGB values with a tolerance
def compare_rgb(rgb1, rgb2, tolerance):
    return all(abs(c1 - c2) <= tolerance for c1, c2 in zip(rgb1, rgb2))

# Function to find the position in the IPF color scheme with the closest RGB value
def find_color_position(rgb, tolerance):
    # Iterate over the IPF color scheme to find the closest RGB value within the given tolerance
    for y in range(ipf_color_scheme.height):
        for x in range(ipf_color_scheme.width):
            color = ipf_color_scheme.getpixel((x, y))
            if compare_rgb(color, rgb, tolerance):
                return x, y
    
    # Return a default position if no close match is found
    return -1, -1

def update_labels(event):
    # Get the coordinates of the cursor on the microstructure image
    x = event.x
    y = event.y
    
    # Get the RGB value of the pixel under the cursor
    rgb = microstructure_image.getpixel((x, y))
    
    # Update the RGB label
    rgb_label.config(text=f"RGB: ({rgb[0]:03d}, {rgb[1]:03d}, {rgb[2]:03d})")
    
    # Find the position in the IPF color scheme with the closest RGB value
    ipf_cursor_x, ipf_cursor_y = find_color_position(rgb, tolerance)
    
    # Update the position of the custom cursor
    canvas.coords(ipf_cursor, microstructure_image.width + 20 + ipf_cursor_x - 5, ipf_cursor_y - 5,
                  microstructure_image.width + 20 + ipf_cursor_x + 5, ipf_cursor_y + 5)
    
    # Calculate the polar coordinates
    polar_radius = math.sqrt((ipf_cursor_x - ipf_color_scheme.width/2)**2 + (ipf_cursor_y - ipf_color_scheme.height/2)**2)
    polar_angle = math.degrees(math.atan2(ipf_cursor_y - ipf_color_scheme.height/2, ipf_cursor_x - ipf_color_scheme.width/2))
    if polar_angle < 0:
        polar_angle += 360
    
    # Update the polar coordinates label
    polar_label.config(text=f"Polar: (Radius: {polar_radius:.2f}, Angle: {polar_angle:.2f} degrees)")

def process():
    # Get the specified RGB values from the entry widgets
    r = int(E1.get())
    g = int(E2.get())
    b = int(E3.get())
    
    # Create a tolerance range for RGB comparison
    tolerance = 20
    
    # Find the position in the IPF color scheme with the specified RGB value
    ipf_cursor_x, ipf_cursor_y = find_color_position((r, g, b), tolerance)
    
    if ipf_cursor_x != -1 and ipf_cursor_y != -1:
        # Update the position of the custom cursor
        canvas.coords(ipf_cursor, microstructure_image.width + 20 + ipf_cursor_x - 5, ipf_cursor_y - 5,
                      microstructure_image.width + 20 + ipf_cursor_x + 5, ipf_cursor_y + 5)
        
        # Calculate the polar coordinates
        polar_radius = math.sqrt((ipf_cursor_x - ipf_color_scheme.width/2)**2 + (ipf_cursor_y - ipf_color_scheme.height/2)**2)
        polar_angle = math.degrees(math.atan2(ipf_cursor_y - ipf_color_scheme.height/2, ipf_cursor_x - ipf_color_scheme.width/2))
        if polar_angle < 0:
            polar_angle += 360
        
        # Update the polar coordinates label
        polar_label.config(text=f"Polar: (Radius: {polar_radius:.2f}, Angle: {polar_angle:.2f} degrees)")

root = tk.Tk()
root.title("Microstructure RGB and Polar Viewer")

# Load the microstructure image and IPF color scheme
microstructure_image_path = r"C:\Users\abhis\Pictures\Screenshots\Picture1.png"
ipf_color_scheme_path = r"C:\Users\abhis\Pictures\Screenshots\IPF1_cropped.png"
microstructure_image = Image.open(microstructure_image_path)
ipf_color_scheme = Image.open(ipf_color_scheme_path)

# Set the tolerance level for color comparison
tolerance = 15

# Create a canvas to display the images
canvas = tk.Canvas(root, width=microstructure_image.width + ipf_color_scheme.width + 20,
                   height=max(microstructure_image.height, ipf_color_scheme.height))
canvas.grid(row=0, column=0, columnspan=2)

# Label to display the RGB value under the cursor
rgb_label = tk.Label(root, font=("Arial", 15), bg="white")
rgb_label.grid(row=1, column=0, columnspan=1, pady=2, padx=90)

# Label to display the polar coordinates of the point on the IPF color scheme
polar_label = tk.Label(root, font=("Arial", 15), bg="white")
polar_label.grid(row=2, column=0, sticky=tk.W, padx=10, pady=10)

# Labels and Entry widgets for specifying RGB values
l1 = tk.Label(root, text=" R ", font=("Arial", 12), bg="white")
l1.grid(row=1, column=1, sticky=tk.W, pady=2)
l2 = tk.Label(root, text=" G ", font=("Arial", 12), bg="white")
l2.grid(row=2, column=1, columnspan=1, sticky=tk.W, pady=2)
l3 = tk.Label(root, text=" B ", font=("Arial", 12), bg="white")
l3.grid(row=3, column=1, columnspan=1, sticky=tk.W, pady=8)

E1 = tk.Entry(root, bd=5)  # Entry widget for R value
E1.grid(row=1, column=1, columnspan=1)
E2 = tk.Entry(root, bd=5)  # Entry widget for G value
E2.grid(row=2, column=1)
E3 = tk.Entry(root, bd=5)  # Entry widget for B value
E3.grid(row=3, column=1)

# Display the microstructure image on the canvas
microstructure_image_photo = ImageTk.PhotoImage(microstructure_image)
canvas.create_image(0, 0, anchor=tk.NW, image=microstructure_image_photo)

# Display the IPF color scheme image on the canvas
ipf_color_scheme_photo = ImageTk.PhotoImage(ipf_color_scheme)
canvas.create_image(microstructure_image.width + 20, 0, anchor=tk.NW, image=ipf_color_scheme_photo)

ipf_cursor = canvas.create_oval(0, 0, 0, 0, outline="black", width=2)

# Bind the "<Motion>" event to the show_rgb_value function
canvas.bind("<Motion>", update_labels)

# Button to initiate the process of finding the specified RGB value
B = tk.Button(root, text="Show Coordinate on IPF Color Key", command=process, bg='yellow')
B.grid(row=5, column=1, sticky=tk.W, padx=25)

root.mainloop()
